In [33]:
import os
import sys
import json
sys.path.append("..")
import pandas as pd
import mlflow
from sklearn.model_selection import cross_validate
import numpy as np
import pandas as pd
from settings import PROJECT_PATH, CLASSIFICATION_TARGET, REGRESSION_TARGET, random_state
from mlflow.tracking import MlflowClient


In [25]:
training_data = pd.read_parquet(os.path.join(PROJECT_PATH, 'data/interim/transactions_post_feature_engineering.parquet'))
training_data.head()

,prix,vefa,n_pieces,surface_habitable,latitude,longitude,mois_transaction,annee_transaction,en_dessous_du_marche,prix_m2_moyen_mois_precedent,nb_transactions_mois_precedent,prix_m2_moyen_glissant_6m,nb_transaction_moyen_glissant_6m,type_batiment_Maison,nom_region_Nouvelle-Aquitaine,nom_region_Occitanie,nom_region_Provence-Alpes-Côte d'Azur,nom_region_Île-de-France
0,30000.0,0,1,34,46.210785,5.228714,6,2018,1,1241.19,11.0,1243.101667,16.666667,0,0,0,0,0
1,73000.0,0,4,77,46.190439,5.238234,6,2018,1,1241.19,11.0,1243.101667,16.666667,0,0,0,0,0
2,185000.0,0,5,107,46.196411,5.274976,6,2018,0,1241.19,11.0,1243.101667,16.666667,1,0,0,0,0
3,65000.0,0,3,65,46.211106,5.219147,6,2018,1,1241.19,11.0,1243.101667,16.666667,0,0,0,0,0
4,84000.0,0,4,86,46.213257,5.228674,6,2018,1,1241.19,11.0,1243.101667,16.666667,0,0,0,0,0


In [26]:
training_data.shape

(468115, 18)

In [27]:
with open ("../preprocessing/features_used/features_name.json", "r") as f :
    features_name = json.load (f)

with open("../preprocessing/features_used/numerical_features.json", "r") as f :
    numerical_features= json.load(f)
print(features_name)
print(numerical_features)

['vefa', 'surface_habitable', 'latitude', 'longitude', 'mois_transaction', 'annee_transaction', 'prix_m2_moyen_mois_precedent', 'nb_transactions_mois_precedent', 'prix_m2_moyen_glissant_6m', 'nb_transaction_moyen_glissant_6m', 'type_batiment_Maison', 'nom_region_Nouvelle-Aquitaine', 'nom_region_Occitanie', "nom_region_Provence-Alpes-Côte d'Azur", 'nom_region_Île-de-France']
['surface_habitable', 'latitude', 'longitude', 'mois_transaction', 'annee_transaction', 'prix_m2_moyen_mois_precedent', 'nb_transactions_mois_precedent', 'prix_m2_moyen_glissant_6m', 'nb_transaction_moyen_glissant_6m']


In [28]:
categorial_features = [col for col in features_name if col not in numerical_features]
print(categorial_features)

['vefa', 'type_batiment_Maison', 'nom_region_Nouvelle-Aquitaine', 'nom_region_Occitanie', "nom_region_Provence-Alpes-Côte d'Azur", 'nom_region_Île-de-France']


In [29]:
X= training_data.drop([REGRESSION_TARGET,CLASSIFICATION_TARGET],axis=1)
y_regression = training_data[REGRESSION_TARGET]
y_classification = training_data[CLASSIFICATION_TARGET]

In [34]:
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

In [37]:
experiment_tags = {
        "prediction_type": "classification",
        "étape": "training",
        "date_de_construction": "Janvier 2025"
    }

In [38]:
try:
        prediction_experiment = client.create_experiment("prediction", tags=experiment_tags)
except:
        prediction_experiment = mlflow.set_experiment("prediction")